In [1]:
import itertools
import random
import numpy as np
import pandas as pd
import math
import time
import simpy
import json
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import tkinter as tk
from PIL import ImageTk
import random
from datetime import datetime, timedelta
import os
from collections import defaultdict

# Input và thiết lập/ràng buộc cho chương trình

In [2]:
######################
##### INPUTS #########
######################
RFID_GATE_LINES = 2
PAPER_GATE_LINES = 2

PAPER_EMPS_PER_LINE = 1
RFID_EMPS_PER_LINE = 0.5

######################
### CONFIGURATIONS ###
######################
RFID_SELECTION_RATE = 0.7

RFID_SCAN_TIME_MIN = 5
RFID_SCAN_TIME_MAX = 12
PAPER_SCAN_TIME_MIN = 10
PAPER_SCAN_TIME_MAX = 17

JOIN_RATE_HIGH_MEAN = 2
JOIN_RATE_HIGH_STD = 0.05

JOIN_RATE_AVG_MEAN = 8
JOIN_RATE_AVG_STD = 1

JOIN_RATE_LOW_MEAN = 60
JOIN_RATE_LOW_STD = 10

ERROR_RATE_RFID = 0.01
ERROR_RATE_PAPER = 0.035
# Creating the 'output' directory if it doesn't exist
os.makedirs('output', exist_ok=True)

# Xây dựng các hàm cho chương trình chính

## Nhóm hàm bổ trợ

In [3]:
def get_current_time(elapsed_seconds):
    start_time = datetime.strptime('06:30:00', '%H:%M:%S')
    current_time = start_time + timedelta(seconds=elapsed_seconds)
    formatted_current_time = current_time.strftime('%H:%M:%S')
    return formatted_current_time

def check_traffic_status(env):
    current_time = get_current_time(env.now)

    if current_time < "08:30:00":
        return "low"
    elif "08:30:00" <= current_time < "09:30:00":
        return "avg"
    elif "09:30:00" <= current_time < "10:30:00":
        return "high"
    elif "10:30:00" <= current_time < "11:30:00":
        return "avg"
    elif "11:30:00" <= current_time < "12:30:00":
        return "high"
    elif "12:30:00" <= current_time < "14:30:00":
        return "low"
    elif "14:30:00" <= current_time < "15:30:00":
        return "avg"
    elif "15:30:00" <= current_time < "16:30:00":
        return "high"
    elif "16:30:00" <= current_time < "18:30:00":
        return "avg"
    elif "18:30:00" <= current_time < "19:30:00":
        return "high"
    else:
        return "low"

## Nhóm hàm sinh thời gian

In [4]:
def generate_scan_time(card_type):
    if card_type == 'RFID':
        return random.uniform(RFID_SCAN_TIME_MIN, RFID_SCAN_TIME_MAX)
    elif card_type == 'paper':
        return random.uniform(PAPER_SCAN_TIME_MIN, PAPER_SCAN_TIME_MAX)
    

def generate_arrival_time(env):
    traffic_status = check_traffic_status(env)
    if traffic_status == 'high':
        arrival_time = max(0, random.normalvariate(JOIN_RATE_HIGH_MEAN, JOIN_RATE_HIGH_MEAN))
    elif traffic_status == 'low':
        arrival_time = max(0, random.normalvariate(JOIN_RATE_LOW_MEAN, JOIN_RATE_LOW_STD))
    else: # NORMAL
        arrival_time = max(0, random.normalvariate(JOIN_RATE_AVG_MEAN, JOIN_RATE_AVG_STD))        
    return arrival_time, traffic_status

    
def generate_error_correction_time(card_type):
    if card_type == 'RFID':
        if RFID_EMPS_PER_LINE == 1:
            ERROR_CORRECTION_TIME = max(0, random.normalvariate(15, 5))
        elif 0.5 < RFID_EMPS_PER_LINE < 1:
            ERROR_CORRECTION_TIME = max(0, random.normalvariate(20, 5))
        else:
            ERROR_CORRECTION_TIME = max(0, random.normalvariate(30, 5))
    elif card_type == 'paper':
        ERROR_CORRECTION_TIME = max(0, random.normalvariate(10, 2))
        
    return ERROR_CORRECTION_TIME

## Nhóm hàm quy trình

In [2]:
event_log = []

def is_error(card_type):
    if card_type == 'RFID':
        return random.random() <= ERROR_RATE_RFID
    elif card_type == 'paper':
        return random.random() <= ERROR_RATE_PAPER
    
def pick_shortest(lines):
    shuffled = list(zip(range(len(lines)), lines))
    random.shuffle(shuffled)
    shortest = shuffled[0][0]
    for i, line in shuffled:
        if len(line.queue) < len(lines[shortest].queue):
            shortest = i
            break
    return lines[shortest], shortest + 1

def logging_events(person, card_type, gate_line, traffic_status, queue_begin, queue_end, scan_begin, scan_end, error_appearance, correction_begin, error_correction_time, correction_end):
    wait = queue_end - queue_begin
    event_log.append({"event": "WAITING TO BE SCANNED", "person": f"id_{person}", "selected line": f"{card_type}_{gate_line}", "traffic status": traffic_status, "begin time": get_current_time(queue_begin), "end time": get_current_time(queue_end), "duration": round(wait, 2)})
    event_log.append({"event": "SCAN TICKET", "person": f"id_{person}", "selected line": f"{card_type}_{gate_line}", "traffic status": traffic_status, "begin time": get_current_time(scan_begin), "end time": get_current_time(scan_end), "duration": round(scan_end - scan_begin, 2)})
    if error_appearance:
        event_log.append({"event": "ERROR OCCURENCE AND CORRECTION", "person": f"id_{person}", "selected line": f"{card_type}_{gate_line}", "traffic status": traffic_status, "begin time": get_current_time(correction_begin), "end time": get_current_time(correction_end), "duration": round(error_correction_time, 2)})

### 2 quá trình chính
- Quá trình tạo phương tiện đến
- Quá trình sử dụng dịch vụ kiểm vé

In [3]:
# QUÁ TRÌNH TẠO PHƯƠNG TIỆN ĐẾN
def vehicle_arrival(env, rfid_gate_lines, paper_gate_lines):
    # Khởi tạo id cho xe đầu tiên
    next_person_id = 0
    
    # Bắt đầu vòng lặp
    while True:
        # Xác định loại thẻ xe này sẽ sử dụng
        card_type = random.choices(['RFID', 'paper'], weights=[RFID_SELECTION_RATE, 1 - RFID_SELECTION_RATE])[0]
        
        # Nếu là sử dụng thẻ từ, các cổng sẽ là cổng từ
        if card_type == 'RFID':
            gate_lines = rfid_gate_lines
            
        # Nếu là sử dụng thẻ giấy, các cổng sẽ là cổng giấy
        else:
            gate_lines = paper_gate_lines
            
        # Tạo ra phương tiện với id, lựa chọn loại thẻ và các cổng họ có thể đi như đã khai báo ở trên
        # Xác định thời gian cần chờ để phương tiện này xuất hiện và trạng thái giao thông tương ứng
        next_arrival, traffic_status = generate_arrival_time(env)
        yield env.timeout(next_arrival) # Ghi nhận thời gian đã trôi qua trong giả lập

        # Phương tiện này sau đó sẽ tham gia quá trình sử dụng dịch vụ kiểm vé
        env.process(using_gate(env, next_person_id, gate_lines, card_type, traffic_status))
        
        # Tạo id cho phương tiện tiếp theo
        next_person_id += 1

# QUÁ TRÌNH SỬ DỤNG DỊCH VỤ KIỂM VÉ
def using_gate(env, person_id, gate_lines, card_type, traffic_status):
    
    # Ghi nhận thời điểm xếp hàng
    queue_begin = env.now
    # Chọn cổng có ít người đang xếp hàng ở đó nhất
    gate_line = pick_shortest(gate_lines)
    with gate_line[0].request() as req:
        # Tạo request được xếp hàng đến lượt "sử dụng tài nguyên (resource)"
        # Tài nguyên ở đây là cổng được chọn
        yield req
        # Đến lượt sử dụng = đã xếp hàng xong
        # Vẫn còn nắm giữ "tài nguyên", tức là vẫn còn ở trong cổng này chưa ra khỏi
        # Ghi nhận thời điểm xếp hàng xong
        queue_end = env.now

    # Bắt đầu "sử dụng tài nguyên (resource)"
    ### SCANNING
        # Sau khi xếp hàng xong, thẻ xe của phương tiện sẽ được quét
        # Ghi nhận thời điểm bắt đầu quét
        scan_begin = env.now
        # Sinh thời gian quét thẻ cho phương tiện này
        scan_time = generate_scan_time(card_type=card_type)
        # Ghi nhận thời gian trôi qua trong giả lập
        yield env.timeout(scan_time)
        # Ghi nhận thời điểm quét thẻ xong
        scan_end = env.now
        
        
    ### ERROR
        # Xác định có xảy ra lỗi hay không
        error_appearance = is_error(card_type)
        # Ghi nhận thời điểm bắt đầu kiểm lỗi
        correction_begin = env.now
        # Nếu có lỗi phát sinh
        if error_appearance:
            # Sinh thời gian sửa lỗi
            error_correction_time = generate_error_correction_time(card_type)
        # Nếu không có lỗi phát sinh
        else:
            # Thời gian kiểm lỗi bằng 0
            error_correction_time = 0
        
        # Ghi nhận thời gian trôi qua trong giả lập
        yield env.timeout(error_correction_time)
        # Ghi nhận thời điểm sửa lỗi xong
        correction_end = env.now
        
        # LƯU CÁC SỰ KIỆN    
        logging_events(person_id, card_type, gate_line[1], traffic_status, queue_begin, queue_end, scan_begin, scan_end, error_appearance, correction_begin, error_correction_time, correction_end)

# Chạy giả lập và ghi nhận events

In [6]:
env = simpy.Environment()

rfid_gate_lines = [simpy.Resource(env, capacity=RFID_EMPS_PER_LINE) for _ in range(RFID_GATE_LINES)]
paper_gate_lines = [simpy.Resource(env, capacity=PAPER_EMPS_PER_LINE) for _ in range(PAPER_GATE_LINES)]
all_gate_lines = [rfid_gate_lines, paper_gate_lines]


env.process(vehicle_arrival(env, rfid_gate_lines, paper_gate_lines))
env.run(until=14*60*60)


# Writing data to a JSON file
with open('output/events.json', 'w') as outfile:
    # input_string = f"""RFID Gates: {RFID_GATE_LINES} | Paper Gates: {PAPER_GATE_LINES} || Paper Employees per Line: {PAPER_EMPS_PER_LINE} | RFID Employees per Line: {RFID_EMPS_PER_LINE}"""
    # config_string = f"""RFID Selection Rate: {RFID_SELECTION_RATE} || RFID Scan Time (Min): {RFID_SCAN_TIME_MIN}| RFID Scan Time (Max): {RFID_SCAN_TIME_MAX} || Paper Scan Time (Min): {PAPER_SCAN_TIME_MIN}| Paper Scan Time (Max): {PAPER_SCAN_TIME_MAX} || Join Rate High Mean: {JOIN_RATE_HIGH_MEAN}| Join Rate High Std: {JOIN_RATE_HIGH_STD} || Join Rate Avg Mean: {JOIN_RATE_AVG_MEAN}| Join Rate Avg Std: {JOIN_RATE_AVG_STD} || Join Rate Low Mean: {JOIN_RATE_LOW_MEAN}| Join Rate Low Std: {JOIN_RATE_LOW_STD} || Error Rate RFID: {ERROR_RATE_RFID}| Error Rate Paper: {ERROR_RATE_PAPER}"""
    json.dump({"RFID GATES": RFID_GATE_LINES, 
               "PAPER GATES": PAPER_GATE_LINES,
               "RFID EMPLOYEES": int(RFID_GATE_LINES * RFID_EMPS_PER_LINE),
               "PAPER EMPLOYEES": PAPER_GATE_LINES * PAPER_EMPS_PER_LINE,
               "events": event_log}, outfile, indent=4)

# Phân tích thời gian chờ

In [7]:
with open('output/events.json', 'r') as f:
    data = json.load(f)
output_df = pd.DataFrame(data)
events_df = pd.json_normalize(output_df['events'])
output_df = pd.concat([output_df.drop('events', axis=1), events_df], axis=1)
output_df['gate_type'] = output_df['selected line'].str[:-2]
output_df.sort_values('person')


,RFID GATES,PAPER GATES,RFID EMPLOYEES,PAPER EMPLOYEES,event,person,selected line,traffic status,begin time,end time,duration,gate_type
0,2,2,1,2,WAITING TO BE SCANNED,id_0,RFID_1,low,06:30:57,06:30:57,0.00,RFID
1,2,2,1,2,SCAN TICKET,id_0,RFID_1,low,06:30:57,06:31:04,6.96,RFID
2,2,2,1,2,WAITING TO BE SCANNED,id_1,RFID_1,low,06:32:05,06:32:05,0.00,RFID
3,2,2,1,2,SCAN TICKET,id_1,RFID_1,low,06:32:05,06:32:15,10.28,RFID
21,2,2,1,2,SCAN TICKET,id_10,RFID_1,low,06:40:08,06:40:14,6.07,RFID
...,...,...,...,...,...,...,...,...,...,...,...,...
1825,2,2,1,2,SCAN TICKET,id_997,paper_1,high,09:45:03,09:45:18,14.78,paper
2096,2,2,1,2,WAITING TO BE SCANNED,id_998,RFID_1,high,09:44:49,09:51:25,396.02,RFID
2097,2,2,1,2,SCAN TICKET,id_998,RFID_1,high,09:51:25,09:51:35,10.49,RFID
2142,2,2,1,2,SCAN TICKET,id_999,RFID_2,high,09:52:36,09:52:45,8.73,RFID


In [8]:
wait_time_df = output_df.groupby(['RFID GATES', 'PAPER GATES', 'RFID EMPLOYEES', 'PAPER EMPLOYEES', 'person', 'traffic status', 'gate_type'])['duration'].sum().reset_index()
wait_time_df = wait_time_df.rename(columns={'duration': 'wait time'})
wait_time_df.to_csv('output/output.csv', index=False)
wait_time_df

,RFID GATES,PAPER GATES,RFID EMPLOYEES,PAPER EMPLOYEES,person,traffic status,gate_type,wait time
0,2,2,1,2,id_0,low,RFID,6.96
1,2,2,1,2,id_1,low,RFID,10.28
2,2,2,1,2,id_10,low,RFID,6.07
3,2,2,1,2,id_100,low,RFID,10.93
4,2,2,1,2,id_1000,high,paper,33.88
...,...,...,...,...,...,...,...,...
9192,2,2,1,2,id_995,high,RFID,399.85
9193,2,2,1,2,id_996,high,RFID,467.29
9194,2,2,1,2,id_997,high,paper,29.29
9195,2,2,1,2,id_998,high,RFID,406.51


In [9]:
wait_time_df.groupby(['RFID GATES', 'PAPER GATES', 'RFID EMPLOYEES', 'PAPER EMPLOYEES', 'gate_type'])['wait time'].mean().reset_index()

,RFID GATES,PAPER GATES,RFID EMPLOYEES,PAPER EMPLOYEES,gate_type,wait time
0,2,2,1,2,RFID,644.249233
1,2,2,1,2,paper,43.792634


In [10]:
wait_time_df.groupby(['RFID GATES', 'PAPER GATES', 'RFID EMPLOYEES', 'PAPER EMPLOYEES', 'traffic status'])['wait time'].mean().reset_index()

,RFID GATES,PAPER GATES,RFID EMPLOYEES,PAPER EMPLOYEES,traffic status,wait time
0,2,2,1,2,avg,166.861953
1,2,2,1,2,high,580.333860
2,2,2,1,2,low,95.066623
